In [1]:
def vec_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label='',ls='-'):
    ax.text(fro[0]+delta[0]/2,fro[1]+delta[1]/3,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0], delta[1], head_width=0.1*scale, head_length=0.15*scale, color=col,length_includes_head=True,ls=ls)

In [1]:
def line_from_by(ax,fro=[0,0],delta=[1,1],scale=1,col='black',label='',ls='-'):
    ax.text(fro[0]+delta[0]/2*scale,fro[1]+delta[1]/3*scale,label,color=col,fontsize=20)
    return ax.arrow(fro[0], fro[1], delta[0]*scale, delta[1]*scale, head_width=0, head_length=0, color=col,length_includes_head=True,ls=ls)

In [2]:
def define_element(geo,ids):
    block = geo.loc[geo['Pkt'] < 0] #emtpy
    for i in ids:
        block = block.append(geo.loc[geo['Pkt'] == i])
    return block

In [1]:
def plot_blocks(blocks):
    fig, ax = plt.subplots()
    for i,block in enumerate(blocks):
        for k in range(len(block)):
            ax.plot([block.iloc[k]['x / m']],[block.iloc[k]['y / m']],marker='o',color='black')
            ax.text(block.iloc[k]['x / m'],block.iloc[k]['y / m']+0.2, str(int(block.iloc[k]['Pkt'])))
        block = block.append(block.iloc[0])#to close the loop
        ax.plot(block['x / m'], block['y / m'],label='Element %i' %(i+1))#,color='black')
    ax.set_xlabel('$x$ / m')
    ax.set_ylabel('$y$ / m')
    ax.legend()
    ax.set(adjustable='box', aspect='equal')
    fig.tight_layout()

In [5]:
def auto_plot_blocks(geometry):
    block1 = define_element(geometry,[1,2,3,4])
    block2 = define_element(geometry,[4,3,6,5])
    block3 = define_element(geometry,[5,6,7,8])
    block4 = define_element(geometry,[8,7,9])
    plot_blocks([block1,block2,block3,block4])

In [ ]:
def block_area(block):
    if len(block) == 3:
        A = np.abs(block.iloc[0]['x / m'] * (block.iloc[1]['y / m'] - block.iloc[2]['y / m']) + 
                   block.iloc[1]['x / m']* (block.iloc[2]['y / m'] - block.iloc[0]['y / m']) + 
                   block.iloc[2]['x / m'] * (block.iloc[0]['y / m'] - block.iloc[1]['y / m']))/2
    elif len(block) == 4:
        A = np.abs((block.iloc[2]['x / m'] - block.iloc[0]['x / m']) * (block.iloc[3]['y / m'] - block.iloc[1]['y / m']) + 
                   (block.iloc[3]['x / m'] - block.iloc[1]['x / m']) * (block.iloc[0]['y / m'] - block.iloc[2]['y / m'])) / 2
    else:
        raise SystemExit('Element neither triangle nor quadrilateral')
    return A

In [6]:
def length(geo,ids):
    a = geo.loc[geo['Pkt']==ids[0]]
    b = geo.loc[geo['Pkt']==ids[1]]
    L = np.sqrt((a.iloc[0]['x / m'] - b.iloc[0]['x / m'])**2 + (a.iloc[0]['y / m'] - b.iloc[0]['y / m'])**2)
    return L

In [7]:
def gravity_vector(data,nr):
    return np.array([0,-data.iloc[nr-1]['G / kN/m']])

In [ ]:
def cohesion(c,geo,from_id,to_id):
    L = length(geo,[from_id,to_id])
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']])
    vec /= np.linalg.norm(vec)
    vec *= L * c
    return vec

In [ ]:
def cohesion_v(c,v,geo,from_id,to_id):
    return -v/np.linalg.norm(v)*c*length(geo,[from_id,to_id])

In [2]:
def line_of_Q(geo,R,from_id,to_id):
    from_node = geo.loc[geo['Pkt']==from_id]
    to_node = geo.loc[geo['Pkt']==to_id]
    vec = R.dot(np.array([to_node.iloc[0]['x / m'] - from_node.iloc[0]['x / m'], to_node.iloc[0]['y / m'] - from_node.iloc[0]['y / m']]))
    vec /= np.linalg.norm(vec)
    return vec

In [3]:
def line_of_Q_v(phi,v,outer=True):
    beta = np.deg2rad(90) + phi
    if outer:
        R = np.array([[np.cos(-beta),-np.sin(-beta)],[np.sin(-beta),np.cos(-beta)]]) #negative
    else:
        R = np.array([[np.cos(beta),-np.sin(beta)],[np.sin(beta),np.cos(beta)]]) #positive
    
    vec = R.dot(v)
    vec /= np.linalg.norm(vec)
    return vec

In [1]:
def get_P_T(eta, *args):
    eta=eta[0]
    geometry = args[0]
    gamma = args[1]
    c=args[2]
    phi=args[3]
    plotfig=args[4]
    #mobilised values
    c /= eta
    phi = np.arctan(np.tan(np.deg2rad(phi))/eta)
    #Achtung: Nummerierung im Umlaufsinn!
    block1 = define_element(geometry,[1,2,3,4])
    block2 = define_element(geometry,[4,3,6,5])
    block3 = define_element(geometry,[5,6,7,8])
    block4 = define_element(geometry,[8,7,9])
    #
    data = {'A / m²': np.array([block_area(block1),block_area(block2),block_area(block3),block_area(block4)])} #in m
    data['G / kN/m'] = data['A / m²']*gamma
    blockdata = pd.DataFrame(data)
    #
    beta = np.deg2rad(90) - phi
    Rp = np.array([[np.cos(beta),-np.sin(beta)],[np.sin(beta),np.cos(beta)]])
    Rn = np.array([[np.cos(-beta),-np.sin(-beta)],[np.sin(-beta),np.cos(-beta)]])
    #
    G_4 = gravity_vector(blockdata,4)
    C_98 = cohesion(c,geometry,9,8)
    C_78 = cohesion(c,geometry,7,8)
    e_Q98 = line_of_Q(geometry,Rp,9,8)
    e_Q78 = line_of_Q(geometry,Rn,7,8)
    b4 = -(G_4 + C_98 + C_78)
    A4 = np.array([e_Q78,e_Q98]).T
    f4 = np.linalg.solve(A4,b4)
    Q_78 = e_Q78 * f4[0]
    Q_98 = e_Q98 * f4[1]
    #
    G_3 = gravity_vector(blockdata,3)
    C_87 = - C_78
    Q_87 = - Q_78
    #
    C_85 = cohesion(c,geometry,8,5)
    C_65 = cohesion(c,geometry,6,5)
    e_Q85 = line_of_Q(geometry,Rp,8,5)
    e_Q65 = line_of_Q(geometry,Rn,6,5)
    b3 = -(G_3 + C_87 + C_85 + C_65 + Q_87)
    A3 = np.array([e_Q85,e_Q65]).T
    f3 = np.linalg.solve(A3,b3)
    Q_85 = e_Q85 * f3[0]
    Q_65 = e_Q65 * f3[1]
    #
    G_2 = gravity_vector(blockdata,2)
    C_56 = - C_65
    Q_56 = - Q_65
    C_54 = cohesion(c,geometry,5,4)
    C_34 = cohesion(c,geometry,3,4)
    e_Q54 = line_of_Q(geometry,Rp,5,4)
    e_Q34 = line_of_Q(geometry,Rn,3,4)
    b2 = -(G_2 + C_56 + C_54 + C_34 + Q_56)
    A2 = np.array([e_Q54,e_Q34]).T
    f2 = np.linalg.solve(A2,b2)
    Q_54 = e_Q54 * f2[0]
    Q_34 = e_Q34 * f2[1]
    #
    G_1 = gravity_vector(blockdata,1)
    C_43 = - C_34
    Q_43 = - Q_34
    C_41 = cohesion(c,geometry,4,1)
    e_Q41 = line_of_Q(geometry,Rp,4,1)
    e_PT = np.array([0,-1])
    b1 = -(G_1 + C_43 + C_41 + Q_43)
    A1 = np.array([e_Q41,e_PT]).T
    f1 = np.linalg.solve(A1,b1)
    Q_41 = e_Q41 * f1[0]
    P_T = e_PT * f1[1]
    #
    if plotfig:
        scale = np.linalg.norm(G_2)
        fig, ax = plt.subplots(figsize=(12,12))
        vec_from_by(ax,delta=G_1,label='$G_1$',scale=scale)
        vec_from_by(ax,fro=G_1,delta=G_2,label='$G_2$',scale=scale)
        vec_from_by(ax,fro=G_1+G_2,delta=G_3,label='$G_3$',scale=scale)
        vec_from_by(ax,fro=G_1+G_2+G_3,delta=G_4,label='$G_4$',scale=scale)
        #
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4,delta=Q_98,label='$Q_{98}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98,delta=C_98,label='$C_{98}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98,delta=Q_85,label='$Q_{85}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85,delta=C_85,label='$C_{85}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85,delta=Q_54,label='$Q_{54}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54,delta=C_54,label='$C_{54}$',scale=scale,col='blue')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54,delta=Q_41,label='$Q_{41}$',scale=scale,col='green')
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54+Q_41,delta=C_41,label='$C_{41}$',scale=scale,col='blue')
        #
        vec_from_by(ax,fro=G_1+G_2+G_3+G_4+Q_98+C_98+Q_85+C_85+Q_54+C_54+Q_41+C_41,delta=P_T,label='$P_\\mathrm{T} = %.2f kN/m$' %(np.linalg.norm(P_T)),scale=scale,col='red')
        ax.set(adjustable='box', aspect='equal')
        plt.axis('off');
        plt.show();
    obj_value = np.linalg.norm(P_T)
    return obj_value

In [2]:
def get_dissipative_safety(geometry,gamma,c,phi):
    phi = np.deg2rad(phi)
    #Achtung: Nummerierung im Umlaufsinn!
    block1 = define_element(geometry,[1,2,3,4])
    block2 = define_element(geometry,[4,3,6,5])
    block3 = define_element(geometry,[5,6,7,8])
    block4 = define_element(geometry,[8,7,9])
    #
    v_14 = line_of_Q(geometry, np.eye(2),1,4)
    e_v45 = line_of_Q(geometry, np.eye(2),4,5)
    e_v58 = line_of_Q(geometry, np.eye(2),5,8)
    e_v89 = line_of_Q(geometry, np.eye(2),8,9)
    e_v34 = line_of_Q(geometry, np.eye(2),3,4)
    e_v65 = line_of_Q(geometry, np.eye(2),6,5)
    e_v78 = line_of_Q(geometry, np.eye(2),7,8)
    b_v = np.append(v_14,np.zeros((4,1)))
    A_v = np.zeros((6,6))
    A_v[0:2,0] = e_v45
    A_v[0:2,1] = e_v34
    #
    A_v[2:4,0] = -e_v45
    A_v[2:4,2] = e_v58
    A_v[2:4,3] = e_v65
    #
    A_v[4:6,2] = -e_v58
    A_v[4:6,4] = e_v89
    A_v[4:6,5] = e_v78
    f_v = np.linalg.solve(A_v,b_v)
    v_45 = f_v[0]*e_v45
    v_34 = f_v[1]*e_v34
    v_58 = f_v[2]*e_v58
    v_65 = f_v[3]*e_v65
    v_89 = f_v[4]*e_v89
    v_78 = f_v[5]*e_v78
    #
    data = {'A / m²': np.array([block_area(block1),block_area(block2),block_area(block3),block_area(block4)])} #in m
    data['G / kN/m'] = data['A / m²']*gamma
    blockdata = pd.DataFrame(data)
    #
    beta = np.deg2rad(90) - phi
    Rp = np.array([[np.cos(beta),-np.sin(beta)],[np.sin(beta),np.cos(beta)]])
    Rn = np.array([[np.cos(-beta),-np.sin(-beta)],[np.sin(-beta),np.cos(-beta)]])
    #
    G_4 = gravity_vector(blockdata,4)
    C_98 = cohesion(c,geometry,9,8)
    C_78 = cohesion(c,geometry,7,8)
    e_Q98 = line_of_Q(geometry,Rp,9,8)
    e_Q78 = line_of_Q(geometry,Rn,7,8)
    b4 = -(G_4 + C_98 + C_78)
    A4 = np.array([e_Q78,e_Q98]).T
    f4 = np.linalg.solve(A4,b4)
    Q_78 = e_Q78 * f4[0]
    Q_98 = e_Q98 * f4[1]
    #
    G_3 = gravity_vector(blockdata,3)
    C_87 = - C_78
    Q_87 = - Q_78
    #
    C_85 = cohesion(c,geometry,8,5)
    C_65 = cohesion(c,geometry,6,5)
    e_Q85 = line_of_Q(geometry,Rp,8,5)
    e_Q65 = line_of_Q(geometry,Rn,6,5)
    b3 = -(G_3 + C_87 + C_85 + C_65 + Q_87)
    A3 = np.array([e_Q85,e_Q65]).T
    f3 = np.linalg.solve(A3,b3)
    Q_85 = e_Q85 * f3[0]
    Q_65 = e_Q65 * f3[1]
    #
    G_2 = gravity_vector(blockdata,2)
    C_56 = - C_65
    Q_56 = - Q_65
    C_54 = cohesion(c,geometry,5,4)
    C_34 = cohesion(c,geometry,3,4)
    e_Q54 = line_of_Q(geometry,Rp,5,4)
    e_Q34 = line_of_Q(geometry,Rn,3,4)
    b2 = -(G_2 + C_56 + C_54 + C_34 + Q_56)
    A2 = np.array([e_Q54,e_Q34]).T
    f2 = np.linalg.solve(A2,b2)
    Q_54 = e_Q54 * f2[0]
    Q_34 = e_Q34 * f2[1]
    #
    G_1 = gravity_vector(blockdata,1)
    C_43 = - C_34
    Q_43 = - Q_34
    C_41 = cohesion(c,geometry,4,1)
    e_Q41 = line_of_Q(geometry,Rp,4,1)
    e_PT = np.array([0,-1])
    b1 = -(G_1 + C_43 + C_41 + Q_43)
    A1 = np.array([e_Q41,e_PT]).T
    f1 = np.linalg.solve(A1,b1)
    Q_41 = e_Q41 * f1[0]
    P_T = e_PT * f1[1]
    #
    P_A = G_1.dot(v_14) + G_2.dot(v_45) + G_3.dot(v_58) + G_4.dot(v_89)
    P_I = np.abs((C_41+Q_41).dot(v_14)) + np.abs((C_54+Q_54).dot(v_45)) + np.abs((C_85+Q_85).dot(v_58)) + np.abs((C_98+Q_98).dot(v_89)) + \
    np.abs((C_43+Q_43).dot(v_34)) + np.abs((C_56+Q_56).dot(v_65)) + np.abs((C_87+Q_87).dot(v_78))
    return P_I/P_A

In [4]:
from ipywidgets import widgets
from ipywidgets import interact

def plot_interactive_failure():
    @interact(iHss=widgets.BoundedFloatText(value=2.0,min=0, step=0.1, description='Höhe Schmierschicht in m'),
              iHslope=widgets.BoundedFloatText(value=6.5,min=1, step=0.1, description='Höhe Böschung in m'),
              ialpha=widgets.BoundedFloatText(value=35,min=10, max=40,step=1, description='Böschungswinkel in °'),
              iphi = widgets.IntSlider(min=20, max=45, value=30, step=5, description='$\\varphi$ / °:'),
              igamma=widgets.IntSlider(min=14, max=25, value=18, step=1, description='$\\gamma$ / kN m$^{-3}$:'),
              iH=widgets.IntSlider(min=1, max=10, value=3, step=1, description='$H$ / m:'),
              ibeta = widgets.IntSlider(min=0, max=15, value=5, step=1, description='$\\beta$ / °:'),
              idelta = widgets.IntSlider(min=0, max=20, value=10, step=2, description='$\\delta_\\mathrm{a}$ / °:'),
             )
    def plot_failure(iHss,iHslope,ialpha,iphi,igamma,iH,ibeta,idelta):
        #unit conversion
        #global c
        #global gamma
        return 0

In [4]:
def get_top_load(geo,p,x):
    fro = x[0]
    if (fro >= geo.loc[geo['Pkt']==4].iloc[0]['x / m']):
        return 0
    to = np.minimum(x[1],geo.loc[geo['Pkt']==4].iloc[0]['x / m'])
    F = p * (to-fro)
    #x_F = fro + (to-fro)/2
    return F#, x_F

SyntaxError: keyword argument repeated (<ipython-input-4-39b3777961df>, line 11)

In [5]:
def get_P_T_wl(eta, *args):
    H_top_slope = args[0]
    H_top_layer = args[1]
    alpha = np.deg2rad(args[2])
    phi1 = np.deg2rad(args[3])
    phi2 = np.deg2rad(args[4])
    c1 = args[5]
    c2 = args[6]
    g1 = args[7]
    g2 = args[8]
    top_load = args[9] #Bemessungswert!
    p_from = args[10]
    p_to = args[11]
    plot = args[12]
    beta = np.deg2rad(args[13])
    theta = np.deg2rad(args[14])

    eta=eta[0]
    
    geometry = pd.DataFrame(np.array([[1],
                                  [0.0],
                                  [H_top_layer]]).T,
                   columns=['Pkt', 'x / m', 'y / m'])
    geometry
    dH = H_top_slope - H_top_layer
    geometry = geometry.append({'Pkt': 2, 'x / m': dH/np.tan(alpha) + dH*np.tan(beta), 'y / m': H_top_layer}, ignore_index=True)
    geometry = geometry.append({'Pkt': 3, 'x / m': dH/np.tan(alpha), 'y / m': H_top_slope}, ignore_index=True)
    geometry = geometry.append({'Pkt': 4, 'x / m': dH/np.tan(alpha) + dH*np.tan(beta) + dH/np.tan(theta), 'y / m': H_top_slope}, ignore_index=True)
    block1 = define_element(geometry,[3,2,4])
    block2 = define_element(geometry,[1,2,3])
    if plot:
        plot_blocks([block1,block2])
    
    v_1 = line_of_Q(geometry, np.eye(2),4,2)
    e_v2 = line_of_Q(geometry, np.eye(2),2,1)
    e_v21 = line_of_Q(geometry, np.eye(2),2,3)
    b_v = v_1
    A_v = np.array([e_v2,-e_v21]).T
    f_v = np.linalg.solve(A_v,b_v)
    v_2 = f_v[0]*e_v2
    v_21 = f_v[1]*e_v21

    material = pd.DataFrame(np.array([['Wichte', 'phi', 'c'],
                                      [g1, np.arctan(np.tan(phi1)/eta), c1/eta],
                                      [g2, np.arctan(np.tan(phi2)/eta), c2/eta]]).T,
                       columns=['Parameter', 'Schluff', 'Schmierschicht'])

    x_shoulder = geometry.loc[geometry['Pkt']==3].iloc[0]['x / m']
    x_load = np.array([x_shoulder + p_from,x_shoulder+p_to]) #Load starts 1m behind shoulder and extends for 5m

    data = {'A / m²': np.array([block_area(block1),block_area(block2)])} #in m
    data['G / kN/m'] = data['A / m²']*float(material.loc[material['Parameter']=='Wichte'].iloc[0]['Schluff'])

    P = np.array([0,-get_top_load(geometry,top_load,x_load)])
    G_1 = gravity_vector(blockdata,1)
    G_2 = gravity_vector(blockdata,2)

    C_1 = cohesion_v(float(material.loc[material['Parameter']=='c'].iloc[0]['Schluff']),v_1,geometry,2,4)
    C_2 = cohesion_v(float(material.loc[material['Parameter']=='c'].iloc[0]['Schmierschicht']),v_2,geometry,1,2)
    C_12 = cohesion_v(float(material.loc[material['Parameter']=='c'].iloc[0]['Schluff']),-v_21,geometry,2,3)
    C_21 = cohesion_v(float(material.loc[material['Parameter']=='c'].iloc[0]['Schluff']),v_21,geometry,2,3)

    e_Q1 = line_of_Q_v(float(material.loc[material['Parameter']=='phi'].iloc[0]['Schluff']),v_1)
    e_Q2 = line_of_Q_v(float(material.loc[material['Parameter']=='phi'].iloc[0]['Schmierschicht']),v_2)
    e_Q12 = line_of_Q_v(float(material.loc[material['Parameter']=='phi'].iloc[0]['Schluff']),-v_21,False)
    e_Q21 = line_of_Q_v(float(material.loc[material['Parameter']=='phi'].iloc[0]['Schluff']),v_21,False)
    e_F = v_2/np.linalg.norm(v_2)

    b_F = np.append(-np.append(G_1+C_1+C_12+P,G_2+C_2+C_21),0)
    A_F = np.zeros((5,5))
    A_F[0:2,0] = e_Q12
    A_F[0:2,1] = e_Q1
    A_F[2:4,2] = e_Q21
    A_F[2:4,3] = e_Q2
    A_F[2:4,4] = e_F
    A_F[4,0] = 1
    A_F[4,2] = -1

    f_F = np.linalg.solve(A_F,b_F)
    f_F

    if (f_F[0:4] < 0).any():
        print("Resultierende Zugkräfte bei Einträgen", np.where(f_F[0:4] < 0))
    #else:
        #print("Keine Zugkräfte an Gleitfugenresultierenden")

    Q_12 = f_F[0]*e_Q12
    Q_1 = f_F[1]*e_Q1
    Q_21 = f_F[2]*e_Q21
    Q_2 = f_F[3]*e_Q2
    Q_1 = f_F[1]*e_Q1

    dF = f_F[4] * e_F

    obj_value = np.linalg.norm(dF)
    return obj_value

IndentationError: unexpected indent (<ipython-input-5-33a01cb24665>, line 94)

In [ ]:
def optimize_mechanism(angles,*args):
    additional = list(args)
    additional.append(angles[0])
    additional.append(angles[1])
    additional = tuple(additional)
    res = sp.optimize.root(fun=get_P_T_wl,x0=1.0,args=additional)
    return res.x